In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow.compat.v1 as tf
import os

import cProfile
from contacts_dataset import *
import parser

/Users/SXMao/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/SXMao/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/SXMao/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/SXMao/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [37]:
tf.enable_eager_execution()

print(tf.executing_eagerly())


features = [
      "profile",
      "hhblits_profile",
      "aatype",
#       "pseudo_frob",
#       "pseudolikelihood",
#       "deletion_probability",
#       "gap_matrix",
#       "pseudo_bias",
#       "profile_with_prior",
#       "profile_with_prior_without_gaps",
#       "reweighted_profile",
#       "non_gapped_profile",
#       "hmm_profile",
#       "num_alignments",
      "seq_length",
#       "num_effective_alignments",
#       "resolution",
#       "sec_structure",
#       "sec_structure_mask",
#       "solv_surf",
#       "solv_surf_mask",
#       "beta_positions",
#       "beta_mask",
#       "domain_name",
#       "chain_name",
#       "resolution",
#       "num_alignments",
#       "superfamily",
#       "profile",
#       "hhblits_profile",
#       "residue_index",
#       "between_segment_residues",
#       "sequence"
    ]

needed_features = [
    'aatype',
    'deletion_probability',
    'gap_matrix',
    'hmm_profile',
#     'mutual_information',
    'non_gapped_profile',
    'num_alignments',
    'profile_with_prior',
    'profile_with_prior_without_gaps',
    'pseudo_bias',
    'pseudo_frob',
    'pseudolikelihood',
    'residue_index',
    'reweighted_profile',
    'seq_length',
    'sequence'
] #TODO

True


In [3]:
FEATURES = {
    'aatype': (tf.float32, [NUM_RES, 21]),
    'alpha_mask': (tf.int64, [NUM_RES, 1]),
    'alpha_positions': (tf.float32, [NUM_RES, 3]),
    'beta_mask': (tf.int64, [NUM_RES, 1]),
    'beta_positions': (tf.float32, [NUM_RES, 3]),
    'between_segment_residues': (tf.int64, [NUM_RES, 1]),
    'chain_name': (tf.string, [1]),
    'deletion_probability': (tf.float32, [NUM_RES, 1]),
    'domain_name': (tf.string, [1]),
    'gap_matrix': (tf.float32, [NUM_RES, NUM_RES, 1]),
    'hhblits_profile': (tf.float32, [NUM_RES, 22]),
    'hmm_profile': (tf.float32, [NUM_RES, 30]),
#     'key': (tf.string, [1]),
    'mutual_information': (tf.float32, [NUM_RES, NUM_RES, 1]),
    'non_gapped_profile': (tf.float32, [NUM_RES, 21]),
    'num_alignments': (tf.int64, [NUM_RES, 1]),
    'num_effective_alignments': (tf.float32, [1]),
    'phi_angles': (tf.float32, [NUM_RES, 1]),
    'phi_mask': (tf.int64, [NUM_RES, 1]),
    'profile': (tf.float32, [NUM_RES, 21]),
    'profile_with_prior': (tf.float32, [NUM_RES, 22]),
    'profile_with_prior_without_gaps': (tf.float32, [NUM_RES, 21]),
    'pseudo_bias': (tf.float32, [NUM_RES, 22]),
    'pseudo_frob': (tf.float32, [NUM_RES, NUM_RES, 1]),
    'pseudolikelihood': (tf.float32, [NUM_RES, NUM_RES, 484]),
    'psi_angles': (tf.float32, [NUM_RES, 1]),
    'psi_mask': (tf.int64, [NUM_RES, 1]),
    'residue_index': (tf.int64, [NUM_RES, 1]),
    'resolution': (tf.float32, [1]),
    'reweighted_profile': (tf.float32, [NUM_RES, 22]),
    'sec_structure': (tf.int64, [NUM_RES, 8]),
    'sec_structure_mask': (tf.int64, [NUM_RES, 1]),
    'seq_length': (tf.int64, [NUM_RES, 1]),
    'sequence': (tf.string, [1]),
    'solv_surf': (tf.float32, [NUM_RES, 1]),
    'solv_surf_mask': (tf.int64, [NUM_RES, 1]),
    'superfamily': (tf.string, [1]),
}

In [4]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


In [57]:
def create_example(data, features=features):
    feature_map = {}
    for f in features:
        print(f)
        if FEATURES.get(f)[0] == tf.float32:
#             print('float32')
            feature_map[f] = _float_feature(data[f].numpy())
        elif FEATURES.get(f)[0] == tf.int64:
#             print('int64')
            feature_map[f] = _int64_feature(data[f].numpy())
        elif FEATURES.get(f)[0] == tf.string:
#             print('string')
            feature_map[f] = _bytes_feature(data[f])
#         print(feature_map)
    example = tf.train.Example(features=tf.train.Features(feature=feature_map))
    return example

In [56]:
features = list(FEATURES.keys())
features

['aatype',
 'alpha_mask',
 'alpha_positions',
 'beta_mask',
 'beta_positions',
 'between_segment_residues',
 'chain_name',
 'deletion_probability',
 'domain_name',
 'gap_matrix',
 'hhblits_profile',
 'hmm_profile',
 'mutual_information',
 'non_gapped_profile',
 'num_alignments',
 'num_effective_alignments',
 'phi_angles',
 'phi_mask',
 'profile',
 'profile_with_prior',
 'profile_with_prior_without_gaps',
 'pseudo_bias',
 'pseudo_frob',
 'pseudolikelihood',
 'psi_angles',
 'psi_mask',
 'residue_index',
 'resolution',
 'reweighted_profile',
 'sec_structure',
 'sec_structure_mask',
 'seq_length',
 'sequence',
 'solv_surf',
 'solv_surf_mask',
 'superfamily']

In [47]:
features

['aatype',
 'alpha_mask',
 'alpha_positions',
 'beta_mask',
 'beta_positions',
 'between_segment_residues',
 'chain_name',
 'deletion_probability',
 'domain_name',
 'gap_matrix',
 'hhblits_profile',
 'hmm_profile',
 'mutual_information',
 'non_gapped_profile',
 'num_alignments',
 'num_effective_alignments',
 'phi_angles',
 'phi_mask',
 'profile',
 'profile_with_prior',
 'profile_with_prior_without_gaps',
 'pseudo_bias',
 'pseudo_frob',
 'pseudolikelihood',
 'psi_angles',
 'psi_mask',
 'residue_index',
 'resolution',
 'reweighted_profile',
 'sec_structure',
 'sec_structure_mask',
 'seq_length',
 'sequence',
 'solv_surf',
 'solv_surf_mask',
 'superfamily']

In [124]:
# features = FEATURES.keys()
tf_dataset = create_tf_dataset('./casp13_data/T0955/T0955.tfrec', features)
# tf_dataset = tf.data.TFRecordDataset(filenames=['./casp13_data/T0965/T0965.tfrec'])
# tf_dataset

In [125]:
new_examples = []
for example in tf_dataset.take(10):
    data = {}
    for f in features:
#         print('num_alignments', example['num_alignments'])
        data[f] = tf.reshape(example[f], [-1])
#         if f not in needed_features:
#             data[f] = tf.zeros(data[f].shape, dtype=FEATURES.get(f)[0])
#     print(data)
    new_example = create_example(data, features=features)
#     print(new_example)
    new_examples.append(new_example)

# new_examples

aatype
alpha_mask
alpha_positions
beta_mask
beta_positions
between_segment_residues
chain_name
deletion_probability
domain_name
gap_matrix
hhblits_profile
hmm_profile
mutual_information
non_gapped_profile
num_alignments
num_effective_alignments
phi_angles
phi_mask
profile
profile_with_prior
profile_with_prior_without_gaps
pseudo_bias
pseudo_frob
pseudolikelihood
psi_angles
psi_mask
residue_index
resolution
reweighted_profile
sec_structure
sec_structure_mask
seq_length
sequence
solv_surf
solv_surf_mask
superfamily


In [145]:
with tf.python_io.TFRecordWriter('./test/T0955/T0955.tfrec') as writer:
    for example in tf_dataset.take(20):
        data = {}
        for f in features:
            data[f] = tf.reshape(example[f], [-1])
            if f not in ['chain_name', 'domain_name'] and f not in needed_features:
                data[f] = tf.zeros(data[f].shape, dtype=FEATURES.get(f)[0])
#         print(data)
        new_example = create_example(data, features=features)
        writer.write(new_example.SerializeToString())

aatype
alpha_mask
alpha_positions
beta_mask
beta_positions
between_segment_residues
chain_name
deletion_probability
domain_name
gap_matrix
hhblits_profile
hmm_profile
mutual_information
non_gapped_profile
num_alignments
num_effective_alignments
phi_angles
phi_mask
profile
profile_with_prior
profile_with_prior_without_gaps
pseudo_bias
pseudo_frob
pseudolikelihood
psi_angles
psi_mask
residue_index
resolution
reweighted_profile
sec_structure
sec_structure_mask
seq_length
sequence
solv_surf
solv_surf_mask
superfamily


In [146]:
test_dataset = create_tf_dataset('./test/T0955/T0955.tfrec', list(FEATURES.keys()))

In [147]:
for example in test_dataset.take(1):
    data = {}
    for f in features:
        data[f] = example[f]
    print(len(data.keys()))

36


In [148]:
for example in tf_dataset.take(1):
    true_data = {}
    for f in features:
        true_data[f] = example[f]
    print(list(true_data.keys()))

['aatype', 'alpha_mask', 'alpha_positions', 'beta_mask', 'beta_positions', 'between_segment_residues', 'chain_name', 'deletion_probability', 'domain_name', 'gap_matrix', 'hhblits_profile', 'hmm_profile', 'mutual_information', 'non_gapped_profile', 'num_alignments', 'num_effective_alignments', 'phi_angles', 'phi_mask', 'profile', 'profile_with_prior', 'profile_with_prior_without_gaps', 'pseudo_bias', 'pseudo_frob', 'pseudolikelihood', 'psi_angles', 'psi_mask', 'residue_index', 'resolution', 'reweighted_profile', 'sec_structure', 'sec_structure_mask', 'seq_length', 'sequence', 'solv_surf', 'solv_surf_mask', 'superfamily']


In [149]:
import numpy as np
for f in features:
#     print(f)
    if not np.array_equal(data[f].numpy(), true_data[f].numpy()):
        print(f)
        print(data[f])
        print(true_data[f])

hhblits_profile
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0.

In [121]:
true_data['hmm_profile']

<tf.Tensor: id=24167, shape=(41, 30), dtype=float32, numpy=
array([[0. , 0. , 0. , ..., 0.5, 1. , 1. ],
       [0. , 0. , 0. , ..., 0.5, 1. , 1. ],
       [0. , 0. , 0. , ..., 0.5, 1. , 1. ],
       ...,
       [0. , 0. , 0. , ..., 0.5, 1. , 1. ],
       [0. , 0. , 0. , ..., 0.5, 1. , 1. ],
       [0. , 1. , 0. , ..., 0.5, 1. , 1. ]], dtype=float32)>